In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain, SequentialChain

# Configurar el modelo OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Paso 1 - Prompt de generación con Chain of Thought
system_template = "Eres un escritor creativo que razona paso a paso."
human_template = (
    "Piensa en pasos lógicos y escribe un texto de 5 líneas "
    "sobre '{topic}' en estilo {style}."
)

system_msg_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_msg_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_msg_prompt, human_msg_prompt])

generate_chain = LLMChain(llm=llm, prompt=chat_prompt, output_key="generated_text")

# Paso 2 - Refinar el texto generado
refine_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Ahora eres un editor que pule textos."),
    HumanMessagePromptTemplate.from_template(
        "Pulir el siguiente texto para mejorar claridad y estilo:\n\n{generated_text}"
    )
])
refine_chain = LLMChain(llm=llm, prompt=refine_prompt, output_key="refined_text")

# Cadena secuencial
chain = SequentialChain(
    chains=[generate_chain, refine_chain],
    input_variables=["topic", "style"],
    output_variables=["generated_text", "refined_text"],
    verbose=True,
)

# Ejecutar el chain
outputs = chain({"topic": "la luna", "style": "poético y evocativo"})
print("Texto generado:\n", outputs["generated_text"])
print("\nTexto refinado:\n", outputs["refined_text"])




> Entering new SequentialChain chain...

> Finished chain.
Texto generado:
 En el vasto lienzo de la noche, la luna asoma,  
un faro de plata que susurra secretos antiguos.  
Sus rayos acarician la piel del mundo dormido,  
tejiendo sombras danzantes entre susurros de viento,  
mientras el tiempo se detiene, atrapado en su abrazo.

Texto refinado:
 En el vasto lienzo de la noche, la luna se asoma,  
un faro de plata que susurra secretos antiguos.  
Sus rayos acarician la piel del mundo dormido,  
tejiendo sombras danzantes entre los susurros del viento,  
mientras el tiempo se detiene, cautivo en su abrazo.
